In [1]:
import os # type: ignore
import pandas as pd # type: ignore
import logging # type: ignore
from pathlib import Path # type: ignore
from datetime import datetime, timedelta # type: ignore
import numpy as np # type: ignore
import random # type: ignore
import recordlinkage # type: ignore
from recordlinkage.datasets import load_febrl4
import fuzzymatcher
from copy import deepcopy # type: ignore

In [2]:
#########################################################################################
# Set any display options and default values                                            #
#########################################################################################
pd.set_option('display.max_row', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)
BASEDIR = 'C:\\Users\\simon\\Documents\\py_projects\\retailnlp\\retailnlp'

In [3]:
# Creates full path and name for an output file - default assumes a log file and will auto create target folder if it doesn't exist
class Filename:
    def __init__(self, module_name='retailnlp', typeofile='log', suffix='.log', folder_name='logs', sep='_', term=''):
        self.module_name = module_name
        self.typeofile = typeofile
        self.suffix = suffix
        self.folder_name = folder_name
        self.sep = sep
        self.term = term
        self.basedir = BASEDIR
        self.filepath = os.path.join(BASEDIR, self.folder_name)
        self.filename = self.module_name + self.sep + self.typeofile + self.sep + self.term + self.sep + self.get_timestamp()
        self.filepathandname = Path(self.filepath, self.filename).with_suffix(self.suffix)
        self.make_dir()
    
    def __str__(self):
        return f'filepathandname:{str(self.filepathandname)}'
    
    def make_dir(self):
        this_filepath = os.path.join(BASEDIR, self.filepath)
        try:
            os.mkdir(this_filepath)
        except FileExistsError:
            pass

    def get_timestamp(self):
        today = datetime.today().strftime("%Y-%m-%d")
        now = datetime.now().strftime("%H-%M-%S")
        return today + '_' + now

# Creates and configures a log
class Log:
    def __init__(self, module_name='retailnlp', term='', level='INFO'):
        self.module_name = module_name
        self.log = logging.getLogger(module_name)
        self.term = term
        self.level = level
        self.log.setLevel(self.level)
        self.log_format = '%(asctime)s - %(name)s - %(levelname)s - %(message)s'
        self.date_format = '%Y-%m-%d %H:%M:%S'
        self.formatter = logging.Formatter(self.log_format, self.date_format)
        self.log_write_mode = 'w+'
        
    @property
    def log_filepathandname(self):
        f = Filename(self.module_name, term=self.term)
        return f.filepathandname
    
    def log_addfh(self):
        self.file_handler = logging.FileHandler(self.log_filepathandname, self.log_write_mode)
        self.file_handler.setFormatter(self.formatter)
        self.log.addHandler(self.file_handler)
    
    def log_addch(self):
        self.console_handler = logging.StreamHandler()
        self.console_handler.setFormatter(self.formatter)
        self.log.addHandler(self.console_handler)

    def log_remove_handlers(self):
        self.log.info('log_remove_handlers: Removing all existing log handlers')
        # get all loggers
        loggers = [logging.getLogger(name) if 'retail' in name else None for name in logging.root.manager.loggerDict]
        # for each valid logger remove all handlers
        for log in loggers:
            if log != None:
                while bool(len(log.handlers)):
                    for handler in log.handlers:
                        print('removing handler!')
                        log.removeHandler(handler)

#Load data from a csv file - create a df
class LoadCSV:
    def __init__(self, filename, projectname='', sep='\t'):
        self.project_name = projectname
        self.filename = filename
        self.sep = sep
    
    @property
    def filepath(self):
        return os.path.join(BASEDIR, 'data', self.filename)
    
    @property
    def df(self):
        df = pd.read_csv(self.filepath, self.sep, header=0, encoding='latin1', engine='python')
        df.columns = df.columns.str.strip().str.upper().str.replace(r'\s+', '_').str.replace('-', '_').str.replace('(', '').str.replace(')', '')
        return df

In [4]:
class Product:
    def __init__(self, product_number, result_page_number, result_page_index_position, product_name, product_description, web_selling_price, calculated_cost_price):
        self.product_number = product_number
        self.result_page_number = result_page_number
        self.result_page_index_position = result_page_index_position 
        self.product_name = product_name
        self.product_description = product_description
        self.web_selling_price = web_selling_price
        self.calculated_cost_price = calculated_cost_price

    def __str__(self):
        return
        f'product_number={self.product_number}:'
        + f'result_page_number={self.result_page_number}:'
        + f'result_page_index_position={self.result_page_index_position}:'
        + f'name={self.product_name}:description{self.product_name}:'
        + f'name={self.product_description}:description{self.product_description}:'
        + f'web_selling_price{self.web_selling_price}:'
        + f'cost_price{self.calculated_cost_price}'

    def to_dict(self):
        return {
            'PRODUCT_NUMBER': self.product_number
            ,'RESULT_PAGE_NUMBER': self.result_page_number
            ,'RESULT_PAGE_INDEX_POSITION': self.result_page_index_position
            ,'PRODUCT_NAME': self.product_name
            ,'PRODUCT_DESCRIPTION': self.product_description
            ,'WEB_SELLING_PRICE': self.web_selling_price
            ,'CALCULATED_COST_PRICE': self.calculated_cost_price
        }

class Invoice:
    def __init__(self, invoice_date):
        self.invoice_number = random.sample(range(1000000),1)[0]
        self.invoice_date = invoice_date
        self.invoice_number_of_lines = random.randint(5,15)
        self.invoice_lines = []
        
    def add_invoice_line(self, product, line_number):
        invoice_line = InvoiceLine(self.invoice_number, self.invoice_date, line_number, product)
        self.invoice_lines.append(invoice_line)
    
    def __str__(self):
        return f'invoice_number={self.invoice_number}:invoice_date={self.invoice_date}:number_invoice_lines={len(self.invoice_lines)}'
    
    def to_dict(self):
        return {
            'INVOICE_NUMBER': self.invoice_number
            ,'INVOICE_DATE': self.invoice_date
            ,'NUMBER_INVOICE_LINES': len(self.invoice_lines)
        }
    
    def df(self, list_of_items):
        return pd.DataFrame.from_records([i.to_dict for i in list_of_items])

class InvoiceLine:
    def __init__(self, invoice_number, invoice_date, invoice_line_number, product, original_product_description):
        self.invoice_number = invoice_number
        self.invoice_date = invoice_date
        self.invoice_line_number = invoice_line_number
        self.product = product
        self.original_product_description = original_product_description
    
    def __str__(self):
        return
        f'invoice_number={self.invoice_number}:self.invoice_date={self.invoice_date}:self.invoice_line_number={self.invoice_line_number}:'
        + f'self.product.name={self.product.name}:self.product.description={self.product.description}:'
        + f'self.product.web_selling_price={self.product.web_selling_price}:'
        + f'self.product.calculated_cost_price={self.product.calculated_cost_price}'
        + f'self.original_product_description={self.original_product_description}'
    
    def to_dict(self):
        return {
            'INVOICE_NUMBER': self.invoice_number
            ,'INVOICE_DATE': self.invoice_date
            ,'INVOICE_LINE_NUMBER': self.invoice_line_number
            ,'INVOICE_LINE_PRODUCT_NAME': self.product.product_name
            ,'INVOICE_LINE_PRODUCT_DESCRIPTION': self.product.product_description
            ,'INVOICE_LINE_PRODUCT_WEB_SELLING_PRICE': self.product.web_selling_price
            ,'INVOICE_LINE_PRODUCT_CALCULATED_COST_PRICE': self.product.calculated_cost_price
            ,'INVOICE_LINE_ORIGINAL_PRODUCT_DESCRIPTION': self.original_product_description
        }

In [11]:
def create_products(df_product_csv):
    main_log.log.info('create_products: starting')
    # Start with the webscraped products dataframe and delete all range priced products
    df_non_range_products = df_product_csv.drop(df_product_csv[df_product_csv['FROM_PRICE'] != df_product_csv['TO_PRICE']].index)
     # Select and rename required columns from web product data
    df_products = df_non_range_products[['RESULT_PAGE_NUMBER', 'RESULT_PAGE_INDEX_POSITION', 'SEARCH_TERM', 'DESCRIPTION', 'FROM_PRICE']].copy()
    df_products = df_products.rename(columns={'SEARCH_TERM': 'PRODUCT_NAME', 'DESCRIPTION': 'PRODUCT_DESCRIPTION', 'FROM_PRICE': 'WEB_SELLING_PRICE'})
    # Create / mimic cost price by discounting selling price
    df_products['CALCULATED_COST_PRICE'] = round(df_products['WEB_SELLING_PRICE'] * np.random.randint(85, 95, len(df_products))/100, 2)
    list_of_products = [
        Product(
            str(row['RESULT_PAGE_NUMBER']) + '-' + str(row['RESULT_PAGE_INDEX_POSITION'])
            ,row['RESULT_PAGE_NUMBER']
            ,row['RESULT_PAGE_INDEX_POSITION']
            ,row['PRODUCT_NAME']
            ,row['PRODUCT_DESCRIPTION']
            ,row['WEB_SELLING_PRICE']
            ,row['CALCULATED_COST_PRICE']
        )
        for index, row in df_products.iterrows()
    ]
    main_log.log.info(f'create_products: ending: {len(list_of_products)} products created')
    return list_of_products

def create_invoices(number_of_products, number_of_invoices=20):
    main_log.log.info('create_invoices: starting')
    # Create empty lists to hold outputs
    list_of_invoices = []
    list_of_invoice_lines = []
    # Iterate for number of invoices
    for i in range(number_of_invoices):
        invoice_date = datetime.now() - timedelta(days=i)
        invoice = Invoice(invoice_date.date())
        main_log.log.debug(f'create_invoices: invoice for date {invoice.invoice_date} created')
        product_indexes = random.sample(range(number_of_products), invoice.invoice_number_of_lines)
        # Iterate for number of invoice lines (a random number created on invoice instantiation)
        for line_number in range(invoice.invoice_number_of_lines):
            this_product = deepcopy(list_of_products[product_indexes[line_number]]) # Copy product entity using random index number ref to list of product entities
            invoice_line = InvoiceLine(invoice.invoice_number, invoice.invoice_date, line_number, this_product, this_product.product_description) 
            invoice.invoice_lines.append(invoice_line)
            # list_of_invoice_lines.append(invoice_line)
            main_log.log.debug(f'create_invoices: invoice line for product {invoice_line.product.product_description} created')
        list_of_invoices.append(invoice)
    main_log.log.info(f'create_invoices: ending: {len(list_of_invoices)} invoices created')
    return list_of_invoices

def modify_invoice_line_product_descriptions(list_of_invoices):
    for invoice in list_of_invoices:
        for invoice_line in invoice.invoice_lines:
            split_description = invoice_line.product.product_description.split(',')
            if len(split_description) == 3:
                invoice_line.product.product_description = split_description[1] + split_description[2]

def fuzzymatch(df_products, df_invoice_lines):
    main_log.log.debug(f'fuzzymatch: starting')
    # Dedupe invoice line products
    df_invoice_lines_fm = df_invoice_lines.copy()
    main_log.log.debug(f'fuzzymatch:len df_invoice_lines_fm before dedupe={len(df_invoice_lines_fm)}')
    df_invoice_lines_fm.drop_duplicates(subset=['INVOICE_LINE_PRODUCT_NAME', 'INVOICE_LINE_PRODUCT_DESCRIPTION'], keep='first', inplace=True)
    save_df_as_csv(df_invoice_lines_fm, term='fm_deduped')
    main_log.log.debug(f'fuzzymatch:len df_invoice_lines_fm after dedupe={len(df_invoice_lines_fm)}')
    left_on = ['PRODUCT_NAME', 'PRODUCT_DESCRIPTION']
    right_on = ['INVOICE_LINE_PRODUCT_NAME', 'INVOICE_LINE_PRODUCT_DESCRIPTION'] 
    matched_results = fuzzymatcher.fuzzy_left_join(df_products, df_invoice_lines_fm, left_on, right_on, left_id_col='PRODUCT_NUMBER', right_id_col = 'INVOICE_NUMBER')
    main_log.log.debug(f'fuzzymatch: len matched_results={len(matched_results)}')
    main_log.log.debug(f'fuzzymatch: ending')
    return matched_results

def recordlink(df_products, df_invoice_lines):
    main_log.log.debug(f'recordlink: starting')
    # Prep data for Record Linkage - df's need to have samne number of rows and have index set
    # df_products.set_index('RESULT_PAGE_NUMBER', 'RESULT_PAGE_INDEX_POSITION')
    df_products.set_index('RESULT_PAGE_NUMBER')
    df_invoice_lines_rl = df_invoice_lines.copy()
    # df_invoice_lines_rl.set_index('INVOICE_NUMBER', 'INVOICE_LINE_NUMBER')
    df_invoice_lines_rl.set_index('INVOICE_NUMBER')
    # Get distinct list of products held in invoice lines
    main_log.log.debug(f'recordlink:len df_invoice_lines_rl before dedupe={len(df_invoice_lines_rl)}')
    df_invoice_lines_rl.drop_duplicates(subset=['INVOICE_LINE_PRODUCT_NAME', 'INVOICE_LINE_PRODUCT_DESCRIPTION'], keep='first', inplace=True)
    main_log.log.debug(f'recordlink:len df_invoice_lines_rl after dedupe={len(df_invoice_lines_rl)}')
    save_df_as_csv(df_invoice_lines_rl, term='rl_deduped')
    # number_additional_rows = len(df_products) - len(df_invoice_lines_rl)
    # main_log.log.debug(f'recordlink: before append rows:number of addtl rows={number_additional_rows}:len of df_invoice_lines_rl={len(df_invoice_lines_rl)}:'
    #     + f'len of df_products={len(df_products)}')
    # for i in range(number_additional_rows):
    #     dummy_value = 'dummy' + str(i)
    #     new_row = pd.Series({'INVOICE_NUMBER': dummy_value, 'INVOICE_DATE': dummy_value, 'INVOICE_LINE_NUMBER': dummy_value, 'INVOICE_LINE_PRODUCT_NAME': dummy_value,                          'INVOICE_LINE_PRODUCT_DESCRIPTION': dummy_value, 'INVOICE_LINE_PRODUCT_WEB_SELLING_PRICE': dummy_value, 'INVOICE_LINE_PRODUCT_CALCULATED_COST_PRICE': dummy_value,          })
    #     df_invoice_lines_rl = df_invoice_lines_rl.append(new_row, ignore_index=True)
    # save_df_as_csv(df_invoice_lines_rl, term='deduped_in_line_dummies')
    # main_log.log.debug(f'recordlink: after append rows:number of addtl rows={number_additional_rows}:len of df_invoice_lines_rl={len(df_invoice_lines_rl)}:'
    #     + f'len of df_products={len(df_products)}')
    
    # Fuzzy match invoice descriptions stetching back x days to those on the 'product file' and rank each match
    indexer = recordlinkage.Index()
    indexer.full()
    # indexer.block(left_on=['NAME'], right_on=['INVOICE_LINE_PRODUCT_NAME'])
    candidate_links = indexer.index(df_products, df_invoice_lines_rl)
    main_log.log.debug(f'recordlink: len candidate_links={len(candidate_links)}')
    compare = recordlinkage.Compare()
    compare.exact('PRODUCT_NAME', 'INVOICE_LINE_PRODUCT_NAME', label='PRODUCT_NAME')
    compare.string('PRODUCT_DESCRIPTION', 'INVOICE_LINE_PRODUCT_DESCRIPTION', method='jarowinkler', threshold=0.7, label='PRODUCT_DESCRIPTION')
    feature_vectors = compare.compute(candidate_links, df_products, df_invoice_lines_rl)
    return feature_vectors

def save_list_as_csv(list_of_items, term=''):
    main_log.log.info(f'save_list_as_csv: number of items being saved={len(list_of_items)}')
    df = pd.DataFrame.from_records([item.to_dict() for item in list_of_items])
    filename = Filename(typeofile='data', suffix='.csv', folder_name='data', sep='_', term=term)
    df.to_csv(filename.filepathandname, index=False, sep=",")
    return df

def save_df_as_csv(df, term=''):
    main_log.log.info(f'save_df_as_csv: number of items being saved={len(df)}')
    filename = Filename(typeofile='data', suffix='.csv', folder_name='data', sep='_', term=term)
    df.to_csv(filename.filepathandname, index=False, sep=",")

def classlist_to_df(list_of_items):
    return pd.DataFrame.from_records([i.to_dict() for i in list_of_items])

In [12]:
# sub_log = Log(module_name=main_log.module_name + '.sub', term='main.sub')
main_log = Log(term='main', level='INFO')
main_log.log_addfh()
main_log.log_addch()
main_log.log.info('################ starting ################')
# Load Product data
product_csv = LoadCSV(projectname='retailnlp', filename='products.csv', sep='\t')
# Create Products
list_of_products = create_products(product_csv.df)
list_of_invoices = create_invoices(len(list_of_products), number_of_invoices=10)
# Modify description of invoice line products to enable fuzzy matching
modify_invoice_line_product_descriptions(list_of_invoices)
list_of_invoice_lines = [invoice_line for invoice in list_of_invoices for invoice_line in invoice.invoice_lines]
# Save as CSV's return df's
df_products = save_list_as_csv(list_of_products, term='products')
df_invoices = save_list_as_csv(list_of_invoices, term='invoices')
df_invoice_lines = save_list_as_csv(list_of_invoice_lines, term='invoice_lines')
# Fuzzy match invoice descriptions stetching back x days to those on the 'product file' and rank each match
matched_results = fuzzymatch(df_products, df_invoice_lines)
# features = recordlink(df_products, df_invoice_lines)
# Close
main_log.log_remove_handlers()

2020-10-30 19:20:39 - retailnlp - INFO - ################ starting ################
2020-10-30 19:20:39 - retailnlp - INFO - create_products: starting
2020-10-30 19:20:39 - retailnlp - INFO - create_products: ending: 105 products created
2020-10-30 19:20:39 - retailnlp - INFO - create_invoices: starting
2020-10-30 19:20:39 - retailnlp - INFO - create_invoices: ending: 10 invoices created
2020-10-30 19:20:39 - retailnlp - INFO - save_list_as_csv: number of items being saved=105
2020-10-30 19:20:39 - retailnlp - INFO - save_list_as_csv: number of items being saved=10
2020-10-30 19:20:39 - retailnlp - INFO - save_list_as_csv: number of items being saved=98
2020-10-30 19:20:39 - retailnlp - INFO - save_df_as_csv: number of items being saved=38
2020-10-30 19:20:39 - retailnlp - INFO - log_remove_handlers: Removing all existing log handlers
removing handler!
removing handler!


In [ ]:
# Fuzzy match invoice descriptions stetching back x days to those on the 'product file' and rank each match
features = fuzzymatch(df_products, df_invoice_lines)

In [ ]:
dfA, dfB = load_febrl4()
dfA.sort_values(by=['rec_id'], inplace=True)
dfB.sort_values(by=['rec_id'], inplace=True)

# print(dfA.columns)
# print(dfA.index)
# print(dfA.sort_values(by=['rec_id'], inplace=True))

# dfA.head().sort_values(by=['rec_id'], inplace=True)
# dfB.head().sort_values(by=['rec-id'], inplace=True)


In [ ]:
dfA.head()

In [ ]:
dfB.head()

In [ ]:
print (len(candidate_links))

In [ ]:

indexer = recordlinkage.Index()
indexer.full()
pairs = indexer.index(dfA, dfB)
indexer = recordlinkage.Index()
indexer.block('given_name')
candidate_links = indexer.index(dfA, dfB)
compare_cl = recordlinkage.Compare()

compare_cl.exact('given_name', 'given_name', label='given_name')
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=0.85, label='surname')
compare_cl.exact('date_of_birth', 'date_of_birth', label='date_of_birth')
compare_cl.exact('suburb', 'suburb', label='suburb')
compare_cl.exact('state', 'state', label='state')
compare_cl.string('address_1', 'address_1', threshold=0.85, label='address_1')

features = compare_cl.compute(candidate_links, dfA, dfB)

In [ ]:
features

In [ ]:
def recordlink(df_products, df_invoice_lines):
    main_log.log.debug(f'recordlink: starting')
    # Prep data for Record Linkage - df's need to have samne number of rows and have index set
    # df_products.set_index('RESULT_PAGE_NUMBER', 'RESULT_PAGE_INDEX_POSITION')
    df_products.set_index('RESULT_PAGE_NUMBER')
    df_invoice_lines_rl = df_invoice_lines.copy()
    # df_invoice_lines_rl.set_index('INVOICE_NUMBER', 'INVOICE_LINE_NUMBER')
    df_invoice_lines_rl.set_index('INVOICE_NUMBER')
    # Get distinct list of products held in invoice lines
    main_log.log.debug(f'recordlink:len df_invoice_lines_rl before dedupe={len(df_invoice_lines_rl)}')
    df_invoice_lines_rl.drop_duplicates(subset=['INVOICE_LINE_PRODUCT_NAME', 'INVOICE_LINE_PRODUCT_DESCRIPTION'], keep='first', inplace=True)
    main_log.log.debug(f'recordlink:len df_invoice_lines_rl after dedupe={len(df_invoice_lines_rl)}')
    save_df_as_csv(df_invoice_lines_rl, term='deduped_in_line')
    number_additional_rows = len(df_products) - len(df_invoice_lines_rl)
    main_log.log.debug(f'recordlink: before append rows:number of addtl rows={number_additional_rows}:len of df_invoice_lines_rl={len(df_invoice_lines_rl)}:'
        + f'len of df_products={len(df_products)}')
    for i in range(number_additional_rows):
        dummy_value = 'dummy' + str(i)
        new_row = pd.Series({'INVOICE_NUMBER': dummy_value, 'INVOICE_DATE': dummy_value, 'INVOICE_LINE_NUMBER': dummy_value, 'INVOICE_LINE_PRODUCT_NAME': dummy_value,                          'INVOICE_LINE_PRODUCT_DESCRIPTION': dummy_value, 'INVOICE_LINE_PRODUCT_WEB_SELLING_PRICE': dummy_value, 'INVOICE_LINE_PRODUCT_CALCULATED_COST_PRICE': dummy_value, })
        df_invoice_lines_rl = df_invoice_lines_rl.append(new_row, ignore_index=True)
    save_df_as_csv(df_invoice_lines_rl, term='deduped_in_line_dummies')
    main_log.log.debug(f'recordlink: after append rows:number of addtl rows={number_additional_rows}:len of df_invoice_lines_rl={len(df_invoice_lines_rl)}:'
        + f'len of df_products={len(df_products)}')
    
    
    
    # Fuzzy match invoice descriptions stetching back x days to those on the 'product file' and rank each match
    indexer = recordlinkage.Index()
    indexer.full()
    # indexer.block(left_on=['NAME'], right_on=['INVOICE_LINE_PRODUCT_NAME'])
    candidate_links = indexer.index(df_products, df_invoice_lines_rl)
    main_log.log.debug(f'recordlink: len candidate_links={len(candidate_links)}')
    compare = recordlinkage.Compare()
    compare.exact('NAME', 'INVOICE_LINE_PRODUCT_NAME', label='PRODUCT NAME')
    compare.string('DESCRIPTION', 'INVOICE_LINE_PRODUCT_DESCRIPTION', method='jarowinkler', threshold=0.7, label='PRODUCT DESCRIPTION')
    feature_vectors = compare.compute(candidate_links, df_products, df_invoice_lines_rl)
    return feature_vectors